# Customer Segmentation with K-Means

*Author: Eda AYDIN*

## Table of Contents

- [Import Libraries](#import-libraries)
- [Exploratory Data Analysis](#exploratory-data-analysis)
- [Data Preprocessing & Feature Engineering](#data-preprocessing--feature-engineering)
- [Encoding / Scaling](#encoding--scaling)
- [Modeling](#modeling)
  - [K-Means](#k-means)
  - [Hierarchical Clustering](#hierarchical-clustering)

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
from scipy import stats
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from yellowbrick.cluster import KElbowVisualizer

c:\Users\choon\anaconda3\envs\python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\choon\anaconda3\envs\python39\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\choon\anaconda3\envs\python39\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",50)
pd.set_option("display.float_format", lambda x: "%.2f" %x)
pd.set_option("display.width",None)
warnings.filterwarnings("ignore")

## Dataset History

Online Retail II isimli veri seti İngiltere merkezli bir perakende şirketinin 01/12/2009 - 09/12/2011 tarihleri arasındaki online satış işlemlerini içeriyor. Şirketin ürün kataloğunda hediyelik eşyalar yer almaktadır ve çoğu müşterisinin toptancı olduğu bilgisi mevcuttur.

<table style="border-collapse: collapse; width: 100%; height: 162px;" border="1">
<tbody>
<tr style="height: 18px;">
<td style="width: 1.58479%; height: 18px;"><strong>Features</strong></td>
<td style="width: 65.0818%; height: 18px;"><strong>A&ccedil;ıklama</strong></td>
</tr>
<tr style="height: 18px;">
<td style="width: 1.58479%; height: 18px;">InvoiceNo</td>
<td style="width: 65.0818%; height: 18px;">Fature Numarası (Eğer bu kod C ile başlıyorsa işlemin iptal edildiğini ifade eder.)</td>
</tr>
<tr style="height: 18px;">
<td style="width: 1.58479%; height: 18px;">StockCode</td>
<td style="width: 65.0818%; height: 18px;">&Uuml;r&uuml;n Kodu (Her bir &uuml;r&uuml;n i&ccedil;in eşsiz)</td>
</tr>
<tr style="height: 18px;">
<td style="width: 1.58479%; height: 18px;">Description</td>
<td style="width: 65.0818%; height: 18px;">&Uuml;r&uuml;n ismi</td>
</tr>
<tr style="height: 18px;">
<td style="width: 1.58479%; height: 18px;">Quantity</td>
<td style="width: 65.0818%; height: 18px;">&Uuml;r&uuml;n adedi (Faturalardaki &uuml;r&uuml;nlerde ka&ccedil;ar tane satıldığı)</td>
</tr>
<tr style="height: 18px;">
<td style="width: 1.58479%; height: 18px;">InvoiceDate</td>
<td style="width: 65.0818%; height: 18px;">Fatura Tarihi</td>
</tr>
<tr style="height: 18px;">
<td style="width: 1.58479%; height: 18px;">UnitPrice</td>
<td style="width: 65.0818%; height: 18px;">Fatura Fiyatı (Stelin)</td>
</tr>
<tr style="height: 18px;">
<td style="width: 1.58479%; height: 18px;">CustomerID</td>
<td style="width: 65.0818%; height: 18px;">Eşsiz m&uuml;şteri numarası</td>
</tr>
<tr style="height: 18px;">
<td style="width: 1.58479%; height: 18px;">Country</td>
<td style="width: 65.0818%; height: 18px;">&Uuml;lke ismi</td>
</tr>
</tbody>
</table>

In [4]:
df_ = pd.read_excel("dataset/online_retail_II.xlsx")
df_.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.00,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.00,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.00,United Kingdom


In [ ]:
df = df_.copy()

## Exploratory Data Analysis

## Data Preprocessing & Feature Engineering

## Encoding / Scaling

## Modeling

### K-Means

### Hierarchical Clustering